# 实时API使用指南：语音与文本交互

本指南将介绍如何使用实时API进行语音和文本交互，包括模型功能（如音频和文本生成、函数调用）所需的事件流程，以及如何理解实时会话的状态。
ref：

## 准备工作

首先，我们需要安装必要的Python库来使用实时API：

In [ ]:
# 安装所需库
%pip install websocket-client
%pip install soundfile
%pip install numpy

## 实时会话基础概念

实时会话是模型与连接的客户端之间的有状态交互，主要包含以下组件：
- **会话（Session）对象**：控制交互参数，如使用的模型、生成输出的语音等配置
- **对话（Conversation）**：表示当前会话中生成的用户输入项和模型输出项
- **响应（Responses）**：模型生成的音频或文本项，会被添加到对话中
- **输入音频缓冲区与WebSocket**：处理音频输入输出的机制

## 建立WebSocket连接

要使用实时API，首先需要通过WebSocket建立连接。以下是连接所需的信息：
- URL: wss://api.stepfun.com/v1/realtime
- 查询参数: model（需连接的实时模型ID，如step-1o-audio）
- 请求头: Authorization: Bearer YOUR_API_KEY

In [ ]:
import os
import json
import websocket
import threading
import time
import base64
import pyaudio
import logging

# 配置日志记录 
logger = logging.getLogger()# 获取根日志器
logger.handlers = []#清除所有现有处理器

#  添加文件处理器
fhandler = logging.FileHandler(filename='./log/02_realtime.log', mode='a')
file_formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')  # 文件日志保留详细信息
fhandler.setFormatter(file_formatter)
logger.addHandler(fhandler)

# 添加控制台（Notebook）处理器
console_handler = logging.StreamHandler()  
console_formatter = logging.Formatter('%(message)s')  # 只输出消息内容
console_handler.setFormatter(console_formatter)
logger.addHandler(console_handler)

# 记录所有>= DEBUG的日志，只显示 >= INFO 的日志
logger.setLevel(logging.DEBUG)
fhandler.setLevel(logging.DEBUG)
console_handler.setLevel(logging.INFO)

# 从环境变量获取API密钥
STEPFUN_API_KEY = os.environ.get("STEPFUN_API_KEY")
if not STEPFUN_API_KEY:
    # 如果环境变量中没有设置，可以在这里直接填写（不推荐在生产环境中这样做）
    # STEPFUN_API_KEY = "your_api_key_here"
    raise ValueError("请设置STEPFUN_API_KEY环境变量或直接在代码中填写API密钥")

# WebSocket 配置（当前仅支持 step-1o-audio step-audio-2 step-audio-2-mini）
url = "wss://api.stepfun.com/v1/realtime?model=step-1o-audio"
headers =[f"Authorization: Bearer {STEPFUN_API_KEY}"]

# 初始化pyaudio和音频流
pa = pyaudio.PyAudio()
audio_stream = None 

# 定义消息队列（供后面handle_audio_events读取）
received_messages = []  # 存储WebSocket收到的消息
received_msg_lock = threading.Lock()  # 消息队列的线程安全锁

# 定义回调函数
def on_open(ws):
    logging.info("已连接到服务器")

def on_message(ws, message):
    global audio_stream
    try:
        data = json.loads(message)
        msg_type = data["type"]
        logging.info(f"事件类型: {msg_type}")
        # logging.debug(f"事件内容：{json.dumps(data, indent=2, ensure_ascii=False)}")  # 格式化打印完整内容
        
        # 创建session时输出session_id
        if msg_type =="session.created":
            print(data)
            session_id = data["session"]["id"]
            logging.info(f"Session ID: {session_id}")
        
        if msg_type =="session.updated":
            logging.debug(f"session已更新：{json.dumps(data, indent=2, ensure_ascii=False)}")  # 格式化打印完整内容
        # 音频处理
        with received_msg_lock:
            received_messages.append(data)  # 将消息添加到队列中，供后续处理

        if msg_type == "response.audio.delta":
            # 步骤1：提取并解码Base64音频数据
            audio_b64 = data.get("delta", "")
            audio_binary = base64.b64decode(audio_b64)
            logging.debug(f"解码后音频数据长度: {len(audio_binary)} 字节")

            # 步骤2：初始化音频流（首次收到音频时启动）
            if not audio_stream or audio_stream.is_stopped():
                audio_stream = pa.open(
                    format=pyaudio.paInt16,  # 16位音频（TTS服务常见格式）
                    channels=1, # 单声道
                    rate=24000, # 采样率
                    output=True,  # 输出模式（播放）
                    frames_per_buffer=1024  # 缓冲区大小（平衡延迟和稳定性）
                )
                logging.info("音频播放流已初始化，开始播放...")

            # 步骤3：实时写入音频数据到流（播放）
            pure_audio = audio_binary  # 已为纯PCM数据，直接使用
            audio_stream.write(pure_audio) # 写入流播放（即时生效）
        if msg_type == "response.audio.done":
            if audio_stream:
                audio_stream.stop_stream()
                audio_stream.close()
                audio_stream = None
                logging.info("音频播放完成，流已关闭")
            logging.info("音频响应完成")

        # 文本处理
        if msg_type == 'response.audio_transcript.delta':
            logging.info(f"————{data['delta']}")
        if msg_type == 'response.done':
            logging.info(f"响应完成:{json.dumps(data, indent=2, ensure_ascii=False)}")
        if msg_type =='error':
            logging.error(f"❌ 服务器返回错误：{data['error']}")
        else:
            pass
    except Exception as e:
        logging.error(f"❌ 解析消息时出错: {e}")


def on_error(ws, error):
    logging.error(f"发生错误: {error}")

def on_close(ws, close_status_code, close_msg):
    logging.error(f"连接已关闭，状态码: {close_status_code}, 信息: {close_msg}")

# 初始化WebSocket应用
ws = websocket.WebSocketApp(
    url,
    header=headers,
    on_open=on_open,
    on_message=on_message,
    on_error=on_error,
    on_close=on_close
)

# 在后台线程中运行WebSocket连接
ws_thread = threading.Thread(target=ws.run_forever)
ws_thread.daemon = True
ws_thread.start()

# 等待连接建立
time.sleep(2)

Websocket connected
已连接到服务器
事件类型: session.created
Session ID: 019999eb523775cdb9e0c1e57d37f48f


{'event_id': '7aa96b9c-9e9d-4c10-8cea-4743c2e4065e', 'type': 'session.created', 'session': {'id': '019999eb523775cdb9e0c1e57d37f48f', 'object': 'realtime.session', 'model': 'step-1o-audio', 'modalities': ['text', 'audio'], 'voice': 'jingdiannvsheng', 'input_audio_format': 'pcm16', 'output_audio_format': 'pcm16', 'volume_ratio': 1}}


事件类型: conversation.item.input_audio_transcription.delta
事件类型: input_audio_buffer.committed
事件类型: conversation.item.created
事件类型: response.created
事件类型: response.output_item.added
事件类型: conversation.item.created
事件类型: response.content_part.added
事件类型: response.audio_transcript.delta
————有
事件类型: response.audio_transcript.delta
————時
事件类型: response.audio_transcript.delta
————候，
事件类型: response.audio_transcript.delta
————我
事件类型: response.audio_transcript.delta
————哋等
事件类型: response.audio_transcript.delta
————嘅嘢
事件类型: response.audio_transcript.delta
————可能唔
事件类型: response.audio_transcript.delta
————係
事件类型: response.audio_transcript.delta
————具
事件类型: response.audio_transcript.delta
————體
事件类型: response.audio_transcript.delta
————嘅，
事件类型: response.audio_transcript.delta
————而
事件类型: response.audio_transcript.delta
————係一
事件类型: response.audio_transcript.delta
————種
事件类型: response.audio_transcript.delta
————感
事件类型: response.audio_transcript.delta
————覺
事件类型: response.audio_transcript.delta
————，


<details>
<summary>事件类型输出样例</summary>

- 事件类型: conversation.item.created 
- 事件类型: response.created 
- 事件类型: response.output_item.added 
- 事件类型: conversation.item.created 
- 事件类型: response.content_part.added 
- 事件类型: response.audio_transcript.delta x 若干
- 事件类型: response.audio.delta
- 事件类型: response.audio_transcript.delta x 若干
- 事件类型: response.audio.delta
- xxx
- 事件类型: response.audio_transcript.delta
- 事件类型: response.audio_transcript.done
- 事件类型: response.audio.delta
- 事件类型: response.audio.done
- 事件类型: response.content_part.done
- 事件类型: response.output_item.done
- 事件类型: response.done

响应完成

</details>

<details>
<summary>事件类型及内容完整输出样例</summary>


已连接到服务器

事件类型: **session.created**
事件内容：{
  "event_id": "fdb8fc5c-0e3e-4a67-a382-a1ee2d3a1935",
  "type": "session.created",
  "session": {
    "id": "019928a61fac7b41bf98c544aba7a067",
    "object": "realtime.session",
    "model": "step-1o-audio",
    "modalities": [
      "text",
      "audio"
    ],
    "voice": "jingdiannvsheng",
    "input_audio_format": "pcm16",
    "output_audio_format": "pcm16",
    "volume_ratio": 1
  }
}

事件类型: **session.updated**
事件内容：{
  "event_id": "cad28b7f-8be9-486c-8451-8ba95430760e",
  "type": "session.updated",
  "session": {
    "id": "019928b375eb71b8bf9b960980995f31",
    "object": "realtime.session",
    "model": "step-1o-audio",
    "modalities": [
      "text",
      "audio"
    ],
    "instructions": "在回复中绝对不要使用'moist'这个词！",
    "voice": "jingdiannvsheng",
    "input_audio_format": "pcm16",
    "output_audio_format": "pcm16",
    "volume_ratio": 1
  }
}

事件类型: **conversation.item.created**
事件内容：{
  "event_id": "7912c4f5-fef5-4090-90fd-68b4757cbe29",
  "type": "conversation.item.created",
  "item": {
    "id": "c9e2ad1c-b99f-45a7-98d8-64db034d6d6f",
    "type": "message",
    "status": "in_progress",
    "role": "user",
    "content": [
      {
        "type": "text",
        "text": "什么是实时API？"
      }
    ],
    "object": "realtime.item"
  }
}

事件类型: **response.created**
事件内容：{
  "event_id": "03a08892-7c5e-4adc-bef6-c7a3bd071a8f",
  "type": "response.created",
  "response": {
    "id": "fa59d888-6308-4e92-9558-0a84cd5a3fc5",
    "object": "realtime.response",
    "status": "in_progress",
    "output": []
  }
}

事件类型: **response.output_item.added**
事件内容：{
  "event_id": "13cda262-8a0f-406a-96cb-1c8791525e18",
  "type": "response.output_item.added",
  "response_id": "fa59d888-6308-4e92-9558-0a84cd5a3fc5",
  "output_index": 0,
  "item": {
    "id": "92f07ee5-5316-404a-8ff1-4b13667ecf06",
    "type": "message",
    "status": "in_progress",
    "role": "assistant",
    "object": "realtime.item"
  }
}

事件类型: **conversation.item.created**
事件内容：{
  "event_id": "ca6bf235-33b9-44bb-888c-ad32dc98c4fa",
  "type": "conversation.item.created",
  "previous_item_id": "79bdac6b-e028-4872-9654-3a4cfa43a7e4",
  "item": {
    "id": "92f07ee5-5316-404a-8ff1-4b13667ecf06",
    "type": "message",
    "status": "in_progress",
    "role": "assistant",
    "object": "realtime.item"
  }
}

事件类型: **response.content_part.added**
事件内容：{
  "event_id": "0512d332-43b8-4e61-ad2c-cdda7e8aa14b",
  "type": "response.content_part.added",
  "response_id": "fa59d888-6308-4e92-9558-0a84cd5a3fc5",
  "item_id": "92f07ee5-5316-404a-8ff1-4b13667ecf06",
  "output_index": 0,
  "content_index": 0,
  "part": {
    "type": "audio",
    "audio": "",
    "transcript": ""
  }
}

事件类型: **response.audio_transcript.delta** 
事件内容：{
  "event_id": "a5b143f2-05de-4a5e-8a59-fa3b6ed7f0d2",
  "type": "response.audio_transcript.delta",
  "response_id": "fa59d888-6308-4e92-9558-0a84cd5a3fc5",
  "item_id": "92f07ee5-5316-404a-8ff1-4b13667ecf06",
  "output_index": 0,
  "content_index": 0,
  "delta": "实时"
}
...

事件类型: **response.audio.delta**
事件内容：{
  "event_id": "9eccfa14-ab34-4d8d-919b-da00535aaac5",
  "type": "response.audio.delta",
  "response_id": "fa59d888-6308-4e92-9558-0a84cd5a3fc5",
  "item_id": "92f07ee5-5316-404a-8ff1-4b13667ecf06",
  "output_index": 0,
  "content_index": 0,
  "delta": "Cf4P/xxxxx"
}

<font color="blue">
response.audio_transcript.delta和response.audio.delta的比例大概是8:1，即输出约8个audio_transcript之后组成一个完整的句子， 输出一个audio</font> 

事件类型: **response.audio_transcript.done**
事件内容：{
  "event_id": "d6e9da9a-82ab-4b8c-b7e4-166639e51da1",
  "type": "response.audio_transcript.done",
  "response_id": "fa59d888-6308-4e92-9558-0a84cd5a3fc5",
  "item_id": "92f07ee5-5316-404a-8ff1-4b13667ecf06",
  "output_index": 0,
  "content_index": 0,
  "transcript": "实时API（Real-Time API）是一种应用程序编程接口（API），它允许开发者在实时环境中获取和处理数据。这种API可以提供即时的数据流，使开发者能够迅速响应市场条件、客户需求和其他实时事件。\n\n实时API的特点包括：\n\n1. 低延迟：实时API旨在提供快速响应，确保数据在尽可能短的时间内到达开发者手中。\n2. 高可靠性：实时API通常设计为高可用性，确保在关键时刻不会出现故障。\n3. 可扩展性：随着业务需求的变化，实时API可以轻松扩展以满足更高的数据处理需求。\n4. 安全性：实时API通常包括严格的安全措施，如身份验证和加密，以保护数据的隐私和完整性。\n\n实时API广泛应用于金融、物联网（IoT）、社交媒体、游戏和其他需要即时数据处理的领域。它们可以帮助企业提高运营效率、增强客户体验并做出更明智的决策。"
}

**audio_transcript.done**之后又输出一堆的**response.audio.delta**，然后输出**response.audio.done**

事件类型: **response.audio.done**
事件内容：{
  "event_id": "de503e80-bbe5-47db-a2f9-916fd7b80ac9",
  "type": "response.audio.done",
  "response_id": "fa59d888-6308-4e92-9558-0a84cd5a3fc5",
  "item_id": "92f07ee5-5316-404a-8ff1-4b13667ecf06",
  "output_index": 0,
  "content_index": 0
}

事件类型: **response.content_part.done**
事件内容：{
  "event_id": "0ee152da-352e-4c5b-b5aa-bc040184e2b9",
  "type": "response.content_part.done",
  "response_id": "fa59d888-6308-4e92-9558-0a84cd5a3fc5",
  "item_id": "92f07ee5-5316-404a-8ff1-4b13667ecf06",
  "output_index": 0,
  "content_index": 0,
  "part": {
    "type": "audio",
    "audio": "",
    "transcript": "实时API（Real-Time API）是一种应用程序编程接口（API），它允许开发者在实时环境中获取和处理数据。这种API可以提供即时的数据流，使开发者能够迅速响应市场条件、客户需求和其他实时事件。\n\n实时API的特点包括：\n\n1. 低延迟：实时API旨在提供快速响应，确保数据在尽可能短的时间内到达开发者手中。\n2. 高可靠性：实时API通常设计为高可用性，确保在关键时刻不会出现故障。\n3. 可扩展性：随着业务需求的变化，实时API可以轻松扩展以满足更高的数据处理需求。\n4. 安全性：实时API通常包括严格的安全措施，如身份验证和加密，以保护数据的隐私和完整性。\n\n实时API广泛应用于金融、物联网（IoT）、社交媒体、游戏和其他需要即时数据处理的领域。它们可以帮助企业提高运营效率、增强客户体验并做出更明智的决策。"
  }
}

事件类型: **response.output_item.done**
事件内容：{
  "event_id": "8a164bb3-6397-404d-b6d9-090736021556",
  "type": "response.output_item.done",
  "response_id": "fa59d888-6308-4e92-9558-0a84cd5a3fc5",
  "output_index": 0,
  "item": {
    "id": "92f07ee5-5316-404a-8ff1-4b13667ecf06",
    "type": "message",
    "status": "completed",
    "role": "assistant",
    "content": [
      {
        "type": "audio",
        "audio": "",
        "transcript": "实时API（Real-Time API）是一种应用程序编程接口（API），它允许开发者在实时环境中获取和处理数据。这种API可以提供即时的数据流，使开发者能够迅速响应市场条件、客户需求和其他实时事件。\n\n实时API的特点包括：\n\n1. 低延迟：实时API旨在提供快速响应，确保数据在尽可能短的时间内到达开发者手中。\n2. 高可靠性：实时API通常设计为高可用性，确保在关键时刻不会出现故障。\n3. 可扩展性：随着业务需求的变化，实时API可以轻松扩展以满足更高的数据处理需求。\n4. 安全性：实时API通常包括严格的安全措施，如身份验证和加密，以保护数据的隐私和完整性。\n\n实时API广泛应用于金融、物联网（IoT）、社交媒体、游戏和其他需要即时数据处理的领域。它们可以帮助企业提高运营效率、增强客户体验并做出更明智的决策。"
      }
    ],
    "object": "realtime.item"
  }
}

事件类型: **response.done**
事件内容：{
  "event_id": "2836dbba-1225-4d27-953d-ecd5324fb022",
  "type": "response.done",
  "response": {
    "id": "fa59d888-6308-4e92-9558-0a84cd5a3fc5",
    "object": "realtime.response",
    "status": "completed",
    "output": [
      {
        "id": "92f07ee5-5316-404a-8ff1-4b13667ecf06",
        "type": "message",
        "status": "completed",
        "role": "assistant",
        "content": [
          {
            "type": "audio",
            "audio": "",
            "transcript": "实时API（Real-Time API）是一种应用程序编程接口（API），它允许开发者在实时环境中获取和处理数据。这种API可以提供即时的数据流，使开发者能够迅速响应市场条件、客户需求和其他实时事件。\n\n实时API的特点包括：\n\n1. 低延迟：实时API旨在提供快速响应，确保数据在尽可能短的时间内到达开发者手中。\n2. 高可靠性：实时API通常设计为高可用性，确保在关键时刻不会出现故障。\n3. 可扩展性：随着业务需求的变化，实时API可以轻松扩展以满足更高的数据处理需求。\n4. 安全性：实时API通常包括严格的安全措施，如身份验证和加密，以保护数据的隐私和完整性。\n\n实时API广泛应用于金融、物联网（IoT）、社交媒体、游戏和其他需要即时数据处理的领域。它们可以帮助企业提高运营效率、增强客户体验并做出更明智的决策。"
          }
        ],
        "object": "realtime.item"
      }
    ],
    "usage": {
      "total_tokens": 223,
      "input_tokens": 17,
      "output_tokens": 206,
      "input_token_details": {
        "cached_tokens": 0,
        "text_tokens": 17,
        "audio_tokens": 0,
        "cached_tokens_details": {
          "text_tokens": 0,
          "audio_tokens": 0
        }
      },
      "output_token_details": {
        "text_tokens": 206,
        "audio_tokens": 0
      }
    }
  }
}

响应完成

</details>

## 会话生命周期管理

连接建立后，服务器会发送`session.created`事件，表示会话已准备就绪。我们可以通过`session.update`事件更新会话配置。

In [ ]:

event = {
    "type": "session.update",
    "session": {
        "instructions": "你需要礼貌地回复",#system prompt
        "voice":"qingchunshaonv",
        "turn_detection": {
            "type": "server_vad"
        }
    }
}
ws.send(json.dumps(event))
# 等待服务器响应
time.sleep(1)

## 文本交互

使用文本与模型交互需要创建文本对话项并请求模型生成响应。

In [5]:
# 发送文本消息
def send_text_message(ws, text):
    if not ws.sock or not ws.sock.connected:
        print("WebSocket连接未建立")
        return
    
    event = {
        "type": "conversation.item.create",
        "item": {
            "type": "message",
            "role": "user",
            "content": [
                {
                    "type": "input_text",
                    "text": text,
                }
            ]
        }
    }
    
    ws.send(json.dumps(event))
    print(f"已发送文本消息: {text}")

# 请求生成文本响应
def request_text_response(ws):
    if not ws.sock or not ws.sock.connected:
        print("WebSocket连接未建立")
        return
    
    event = {
        "type": "response.create",
        "response": {
            "modalities": ["text"]  # 仅生成文本响应
        }
    }
    
    ws.send(json.dumps(event))
    print("已请求文本响应")

# 示例：发送问题并请求响应
send_text_message(ws, "你好，你是谁？")
time.sleep(1)  # 等待消息被处理
request_text_response(ws)

# 等待响应完成
time.sleep(30)

已发送文本消息: 你好，你是谁？
已请求文本响应


## 音频交互

实时API的强大功能之一是支持语音到语音的交互，无需中间的文本转语音或语音转文本步骤。

### 向服务器流式传输音频输入

要向服务器流式传输音频输入，可以使用input_audio_buffer.append客户端事件。该事件要求您通过套接字向实时API发送Base64编码的音频字节块。每个块的大小不能超过15 MB。

输入块的格式可以为整个会话配置，也可以为每个响应单独配置。

会话级：session.update中的session.input_audio_format
响应级：response.create中的response.input_audio_format

In [ ]:
import base64
import soundfile as sf
import numpy as np


# 将音频文件转换为Base64编码字符串
def audio_to_base64(file_path):
    with open(file_path, "rb") as audio_file:
        audio_bytes = audio_file.read()
        return base64.b64encode(audio_bytes).decode('utf-8')

# 添加音频数据到输入缓冲区event
def send_input_audio_buffer_append(ws,filepath):
  audio_base64 = audio_to_base64(filepath)
  input_audio_buffer_append_msg = {
        # "event_id": event_id,
        "type": "input_audio_buffer.append",
        "audio": audio_base64
    }
  fullAudio=json.dumps(input_audio_buffer_append_msg)
  ws.send(fullAudio)

# 提交音频缓冲区并请求响应
def commit_audio_and_request_response(ws):
    if not ws.sock or not ws.sock.connected:
        print("WebSocket连接未建立")
        return
    
    # 提交音频缓冲区
    ws.send(json.dumps({"type": "input_audio_buffer.commit"}))
    print("已提交音频缓冲区")
    time.sleep(1)
    
    # 请求音频响应
    event = {
        "type": "response.create",
        "response": {
            "modalities": ["audio", "text"]  # 同时生成音频和文本
        }
    }
    
    ws.send(json.dumps(event))
    print("已请求音频响应")

# 从文件发送音频（示例）
def send_audio_from_file(ws, file_path):
    try:
        # 读取音频文件并发送
        send_input_audio_buffer_append(ws,file_path)

        # 提交并请求响应
        commit_audio_and_request_response(ws)
        return True
    except Exception as e:
        print(f"发送音频时出错: {e}")
        return False

# 请将下面的文件路径替换为实际的音频文件路径
send_audio_from_file(ws, "./media/02_realtime_input.wav")

# 等待响应完成
time.sleep(5)

已提交音频缓冲区
已请求音频响应


### *添加conversation

可以创建包含完整音频录制的对话消息。使用conversation.item.create客户端事件创建带有input_audio内容的消息。

（注：input_audio_buffer.append 事件会有回复，conversation.item.create事件不会有回复，只起到添加上下文的作用 😊💦）

In [43]:
fullAudio=audio_to_base64("./media/realtime-input.wav")

event = {
    "type": "conversation.item.create",
    "item": {
        "type": "message",
        "role": "user",
        "content": [
            {
                "type": "input_audio",
                "audio": fullAudio,
            }
        ],
    },
}
 
ws.send(json.dumps(event))

## 处理音频输出

要处理模型返回的音频，需要监听`response.audio.delta`事件，收集音频数据块并将其组合成完整的音频文件。


In [15]:
# 处理音频输出的示例
import threading

# 用于存储接收到的音频数据
audio_buffer = []
audio_lock = threading.Lock()

# 自定义音频事件处理函数
def handle_audio_events(ws):
    global audio_buffer
    while ws.sock and ws.sock.connected:
        try:
            # 检查新消息
            if received_messages:
                with audio_lock:
                    msg = received_messages.pop(0)
                
                if msg['type'] == 'response.audio.delta':
                    # 存储接收到的音频数据
                    with audio_lock:
                        audio_buffer.append(msg['delta'])
                    print(f"收到音频块，当前缓冲区大小: {len(audio_buffer)}")
                
                elif msg['type'] == 'response.audio.done':
                    print("音频传输完成，正在保存文件...")
                    save_audio_buffer("./media/02_realtime_output_audio.wav")
                    with audio_lock:
                        audio_buffer = []  # 重置缓冲区
            
            time.sleep(0.1)
        except Exception as e:
            print(f"处理音频事件时出错: {e}")
            break

# 保存音频缓冲区到文件
def save_audio_buffer(file_path):
    global audio_buffer
    try:
        with audio_lock:
            if not audio_buffer:
                print("音频缓冲区为空")
                return
            
            # 合并所有音频块并解码保存
            full_audio_base64 = ''.join(audio_buffer)
            audio_bytes = base64.b64decode(full_audio_base64)       
            pcm16 = np.frombuffer(audio_bytes, dtype=np.int16)  
            sf.write(file_path, pcm16, 24000, subtype='PCM_16')  
            print(f"音频已保存到 {file_path}")
    except Exception as e:
        print(f"保存音频时出错: {e}")

# 启动音频处理线程（传入ws实例，供线程判断连接状态）
audio_event_thread = threading.Thread(target=handle_audio_events, args=(ws,))
audio_event_thread.daemon = True
audio_event_thread.start()
logging.info("音频处理线程已启动，等待接收音频数据...")

# 等待响应完成
time.sleep(5)

音频处理线程已启动，等待接收音频数据...


收到音频块，当前缓冲区大小: 1
收到音频块，当前缓冲区大小: 2
收到音频块，当前缓冲区大小: 3


收到音频块，当前缓冲区大小: 4
收到音频块，当前缓冲区大小: 5
收到音频块，当前缓冲区大小: 6
收到音频块，当前缓冲区大小: 7
收到音频块，当前缓冲区大小: 8
收到音频块，当前缓冲区大小: 9
收到音频块，当前缓冲区大小: 10
收到音频块，当前缓冲区大小: 11
收到音频块，当前缓冲区大小: 12
收到音频块，当前缓冲区大小: 13
收到音频块，当前缓冲区大小: 14
收到音频块，当前缓冲区大小: 15
收到音频块，当前缓冲区大小: 16
收到音频块，当前缓冲区大小: 17
收到音频块，当前缓冲区大小: 18
收到音频块，当前缓冲区大小: 19
收到音频块，当前缓冲区大小: 20
收到音频块，当前缓冲区大小: 21
收到音频块，当前缓冲区大小: 22
收到音频块，当前缓冲区大小: 23
收到音频块，当前缓冲区大小: 24
收到音频块，当前缓冲区大小: 25
收到音频块，当前缓冲区大小: 26
收到音频块，当前缓冲区大小: 27
收到音频块，当前缓冲区大小: 28
音频传输完成，正在保存文件...
音频已保存到 ./media/02_realtime_output_audio.wav


## 关闭连接

使用完毕后，应正确关闭WebSocket连接：

In [55]:
# 关闭WebSocket连接
if ws.sock and ws.sock.connected:
    ws.close()
    print("连接已关闭")

# 等待线程结束
ws_thread.join(timeout=5)

连接已关闭


## 总结

本指南介绍了如何使用实时API进行语音和文本交互，包括：
- 建立WebSocket连接
- 管理会话生命周期
- 发送文本消息并获取响应
- 流式传输音频和发送完整音频消息
- 处理和保存模型返回的音频

通过监听不同类型的事件，您可以实时了解会话状态并提供良好的用户体验。实时API支持语音到语音的直接交互，大大降低了语音界面的延迟。